<a href="https://colab.research.google.com/github/99kenny/99kenny/blob/main/TSP_GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<hr style="border:2px solid gray"> </hr>

# Homework 1 - Traveling Salesman Problem

## Example Code

### Algorithm 4: Genetic Algorithm 

### Author: Wangduk Seo (CAU AI Lab)
<hr style="border:2px solid gray"> </hr>

# Step 0. Importing packages and Global Settings

---------------------------------------------------------------
## (Optional) For Colab

In [164]:
from google.colab import drive
import os, sys
drive.mount('gdrive', force_remount=True)

Mounted at gdrive


---------------------------------------------------------------

In [165]:
# package list
import numpy as np
import sys
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib.pyplot as plt
import time

# Global Variables
# GA
POOL_SIZE = 5
TOURNAMENT_SIZE = 5
MAX_ITERATION = 500

# SA
MAX_EVALUATION = 10
SUB_ITERATIONS = 10
TEMPERATURE = 100
COOLING_RATIO = 0.5
TEMP_LIMIT = 1

np.random.seed(0)
# Plot Settings
PLOT_MODE = True # Draw Route
plt.ion()

# First City Index
FIRST_IDX = 0

In [166]:
file_path = '/content/gdrive/MyDrive/hw7/fi10639.txt'

# Step 1. Data Loading

In [167]:
def fileloader():
    #     Data Format
    #     ---------------------------------------------------------
    #     NAME : pia3056
    #     COMMENT : Bonn VLSI data set with 3056 points
    #     COMMENT : Uni Bonn, Research Institute for Discrete Math
    #     COMMENT : Contributed by Andre Rohe
    #     TYPE : TSP
    #     DIMENSION : 3056 -----------------------------|
    #     EDGE_WEIGHT_TYPE : EUC_2D                     |
    #     NODE_COORD_SECTION                            |
    #     1 0 11 (2 dimentional coordinate of city)     |
    #     2 0 115                                       |
    #     ...                                           |
    #     ...(Total 3056 nodes)<------------------------|
    #     EOF
    #     ---------------------------------------------------------
    with open(file_path, "r") as file:
        file_str = file.readlines()

    # Get the coordinates of cities
    coord_str = file_str[8:-1]  # first city string to last city string (EOF 전까지)
    coord_list = np.zeros((len(coord_str), 2))
    for idx, item in enumerate(coord_str):
        items = item.split()
        coord_list[idx, 0], coord_list[idx, 1] = float(items[1]), float(items[2])

    return coord_list

# Step 2. Initialization

In [168]:
def initialize_greedy(coord_list, path_map, first_idx):
    cnt_cities = len(coord_list)
    # Initialize path and insert first city index to the first and last elements
    path = np.zeros(cnt_cities + 1, dtype=np.int32)
    path[0], path[-1] = first_idx, first_idx

    cities_tovisit = np.ones((cnt_cities), dtype=np.bool_)
    cities_tovisit[first_idx] = False

    # Iteratively Connect nearest cities
    for i in range(1, cnt_cities):
        start_idx = path[i - 1]
        distance_from_start = path_map[start_idx, :]
        nearest_list = np.argsort(distance_from_start)
        for idx in range(len(nearest_list)):
            # check the nearest city is visited
            if cities_tovisit[nearest_list[idx]]:
                nearest_city = nearest_list[idx]
                break
        cities_tovisit[nearest_city] = False
        path[i] = nearest_city

    return path



def initialize_random(path, path_map, coord_list):
    cnt_cities = len(path) - 1
    radomized_path = np.zeros(cnt_cities + 1, dtype=np.int32)

    radomized_path[0], radomized_path[-1], radomized_path[-2] = path[0], path[-1], path[-2]
    cities_set = set(path)
    cities_set.remove(path[0])
    cities_set.remove(path[-2])
    
    # city indices without first city index
    cities_tovisit = list(cities_set)
    cities_random = np.random.permutation(cities_tovisit)
    radomized_path[1:-2] = cities_random
    return radomized_path

def path_cost(path_map, path):
    # The array of cost between cities in the path
    cnt_cities = len(path) - 1
    cost_arr = np.zeros(cnt_cities)
    for i in range(cnt_cities):
        cost_arr[i] = path_map[path[i], path[i+1]]

    return cost_arr

# Step 3. Searching a path

## Algorithm 4. Genetic Algorithm 

In [169]:
def two_opt_swap(path_map, path, iterations, coord_list):
    cnt_cities = len(path) - 1
    # Save the best path

    cost_arr = path_cost(path_map, path)
    best_path = path.copy()
    best_cost = cost_arr.sum()
    
    for i in range(iterations):
        curr_path = best_path.copy()
        # Select two indices of flip points
        sel_idx = np.sort(np.random.choice(np.arange(1, cnt_cities), 2))

        # Path Flip and update cost array
        curr_path[sel_idx[0]:sel_idx[1]] = np.flip(curr_path[sel_idx[0]: sel_idx[1]])
        cost_arr = path_cost(path_map, curr_path)

        # Compare to the best path
        curr_cost = cost_arr.sum()
        if curr_cost < best_cost:
            best_path = curr_path
            best_cost = curr_cost
    
    temperature = TEMPERATURE
    while temperature > TEMP_LIMIT:
        curr_path = best_path.copy()
        # Select two indices of flip points
        sel_idx = np.sort(np.random.choice(np.arange(1, cnt_cities), 2))

        # Path Flip and update cost array
        curr_path[sel_idx[0]:sel_idx[1]] = np.flip(curr_path[sel_idx[0]: sel_idx[1]])
        cost_arr = path_cost(path_map, curr_path)
        curr_cost = cost_arr.sum()

        if curr_cost <= best_cost:
            best_path, best_cost = curr_path, curr_cost
        else:
            prob = 1 / np.exp((curr_cost - best_cost) / float(temperature))
            if prob > np.random.rand(1):
                best_path, best_cost = curr_path, curr_cost
        temperature = temperature * COOLING_RATIO 
    return best_path, best_cost

In [170]:
def sa(path_map, path, coord_list):
    best_path, best_cost = path.copy() , path_cost(path_map, path).sum()
    global TEMPERATURE
  
    for i in range(MAX_EVALUATION):
        curr_path = best_path.copy()
        new_path, new_cost = two_opt_swap(path_map, curr_path, SUB_ITERATIONS, coord_list)

        if new_cost < best_cost:
          best_path, best_cost = new_path, new_cost

    return best_path, best_cost

In [171]:
def initialization(path,path_map):
    cnt_cities = len(path) - 1
    path_pool = np.zeros((POOL_SIZE, cnt_cities + 1), dtype=np.int32)
    pool_cost = np.zeros(POOL_SIZE)
    
    path_pool[0, :] = path
    pool_cost[0] = path_cost(path_map, path_pool[0, :]).sum()
    print('Path {} is initialized'.format(0))

    for i in range(1, POOL_SIZE):
        path_pool[i, :] = initialize_random(path, path_map, coord_list)
        path_pool[i, :], pool_cost[i] = sa(path_map, path_pool[i, :], coord_list)
        print('Path {} is initialized'.format(i))
    
    return path_pool, pool_cost

In [172]:
def selection(pool_cost, TOURNAMENT_SIZE, sel_size=2):
    # tournament selection
    sel_idx = np.random.choice(POOL_SIZE, TOURNAMENT_SIZE, replace=False)
    sel_cost = pool_cost[sel_idx]
    best_idx = sel_idx[np.argsort(sel_cost)][:sel_size]
    return best_idx

In [173]:
# pmx crossover
def crossover(path1, path2):
    cnt_cities = len(path1) - 1
    # Select two indices of crossover points
    sel_idx = np.sort(np.random.choice(np.arange(1, cnt_cities), 2))
    # Initialize child path
    child_path = np.zeros(cnt_cities + 1, dtype=np.int32)
    child_path[0], child_path[-1], child_path[-2] = -1, -1, path1[-2]
    visited_cities = set(path1[sel_idx[0]:sel_idx[1]])
    visited_cities.add(path1[0])
    visited_cities.add(path1[-1])
    visited_cities.add(path1[-2])
    # Copy the path between crossover points
    child_path[sel_idx[0]:sel_idx[1]] = path1[sel_idx[0]:sel_idx[1]]
    # Copy the rest of the path from path2
    path2_idx = np.where(np.isin(path2, list(visited_cities)) == False)[0]
    child_path[np.where(child_path == 0)[0]] = path2[path2_idx]
    child_path[0], child_path[-1] = path1[0], path1[-1]

    return child_path


In [174]:
# swap mutation
def mutation(path):
    cnt_cities = len(path) - 1
    child_path = path.copy()
    # Select two indices of mutation points
    sel_idx = np.sort(np.random.choice(np.arange(1, cnt_cities), 2))
    # Swap the path between mutation points
    child_path[sel_idx[0]:sel_idx[1]] = np.flip(child_path[sel_idx[0]:sel_idx[1]])

    return child_path 


In [175]:
# genetic algorithm
def ga(path, coord_list, path_map):
    best_cost = np.Inf
    print('Start Genetic Algorithm')
    print('Initialize the population')
    path_pool, pool_cost = initialization(path, path_map)
    print('Start the evolution')
    for i in range(MAX_ITERATION):
        if (i+1) % 1000 == 0:
            print('Iteration {}'.format(i + 1))
        # selection
        sel_idx = selection(pool_cost, TOURNAMENT_SIZE)
        # crossover
        child_crx = crossover(path_pool[sel_idx[0]], path_pool[sel_idx[1]])
        cost_crx = path_cost(path_map, child_crx).sum()
        
        # mutation
        sel_idx = selection(pool_cost, TOURNAMENT_SIZE, sel_size=1)
        child_mut = mutation(path_pool[sel_idx[0]])
        cost_mut = path_cost(path_map, child_mut).sum()
        # replace
        sort_idx = np.argsort(pool_cost)

        path_pool[sort_idx[-1]], pool_cost[sort_idx[-1]] = child_crx, cost_crx 
        path_pool[sort_idx[-2]], pool_cost[sort_idx[-2]] = child_mut, cost_mut 

        cur_idx = np.argmin(pool_cost)
        cur_path = path_pool[cur_idx]
        cur_cost = pool_cost[cur_idx]

    best_idx = np.argmin(pool_cost)
    return path_pool[best_idx], pool_cost[best_idx]

In [176]:
def dnq(path_map, path, coord_list):
  size = 256
  cnt_cities = len(coord_list)
  sub_tours = []
  sub_tour_len = cnt_cities // size
  last_sub_tour_len = sub_tour_len + (cnt_cities % size)
  #size개의 sub tour
  for i in range(size):
    if i != size-1 :
      index = sub_tour_len
    else:
      index = last_sub_tour_len
    sub_tours.append(path.tolist()[i*sub_tour_len:i*sub_tour_len+index])
  new_path = []
  
  for j in range(size):
    sub_tours[j].append(sub_tours[j][0])
    best_path, best_cost = ga(np.array(sub_tours[j]), coord_list, path_map)
    new_path = new_path + (best_path.tolist()[0:-1])
   
  new_path.append(new_path[0])
  new_cost = path_cost(path_map, np.array(new_path)).sum()
  return new_path, new_cost

# Main

In [177]:
# Step 1
try:
    coord_list = fileloader()
except Exception as e:
    print('예외 발생', e)
    sys.exit()

start_time = time.time()
path_map = euclidean_distances(coord_list, coord_list)
path = initialize_greedy(coord_list, path_map, FIRST_IDX)
best_path, best_cost = dnq(path_map, path, coord_list)

print('Execution Time: ' + str(time.time() - start_time))
print('Path: ' + str(best_path))
print('Cost: ' + str(best_cost))

Start Genetic Algorithm
Initialize the population
Path 0 is initialized
Path 1 is initialized
Path 2 is initialized
Path 3 is initialized
Path 4 is initialized
Start the evolution
Start Genetic Algorithm
Initialize the population
Path 0 is initialized
Path 1 is initialized
Path 2 is initialized
Path 3 is initialized
Path 4 is initialized
Start the evolution
Start Genetic Algorithm
Initialize the population
Path 0 is initialized
Path 1 is initialized
Path 2 is initialized
Path 3 is initialized
Path 4 is initialized
Start the evolution
Start Genetic Algorithm
Initialize the population
Path 0 is initialized
Path 1 is initialized
Path 2 is initialized
Path 3 is initialized
Path 4 is initialized
Start the evolution
Start Genetic Algorithm
Initialize the population
Path 0 is initialized
Path 1 is initialized
Path 2 is initialized
Path 3 is initialized
Path 4 is initialized
Start the evolution
Start Genetic Algorithm
Initialize the population
Path 0 is initialized
Path 1 is initialized
Path 2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: overflow encountered in exp


Path 2 is initialized
Path 3 is initialized
Path 4 is initialized
Start the evolution
Start Genetic Algorithm
Initialize the population
Path 0 is initialized
Path 1 is initialized
Path 2 is initialized
Path 3 is initialized
Path 4 is initialized
Start the evolution
Start Genetic Algorithm
Initialize the population
Path 0 is initialized
Path 1 is initialized
Path 2 is initialized
Path 3 is initialized
Path 4 is initialized
Start the evolution
Start Genetic Algorithm
Initialize the population
Path 0 is initialized
Path 1 is initialized
Path 2 is initialized
Path 3 is initialized
Path 4 is initialized
Start the evolution
Start Genetic Algorithm
Initialize the population
Path 0 is initialized
Path 1 is initialized
Path 2 is initialized
Path 3 is initialized
Path 4 is initialized
Start the evolution
Start Genetic Algorithm
Initialize the population
Path 0 is initialized
Path 1 is initialized
Path 2 is initialized
Path 3 is initialized
Path 4 is initialized
Start the evolution
Start Genetic 